<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Database-functions" data-toc-modified-id="Database-functions-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Database functions</a></span></li><li><span><a href="#Comparator-functions" data-toc-modified-id="Comparator-functions-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Comparator functions</a></span></li><li><span><a href="#Functions-for-final-code" data-toc-modified-id="Functions-for-final-code-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Functions for final code</a></span></li><li><span><a href="#Final-Code" data-toc-modified-id="Final-Code-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Final Code</a></span><ul class="toc-item"><li><span><a href="#CPU" data-toc-modified-id="CPU-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>CPU</a></span></li><li><span><a href="#GPU" data-toc-modified-id="GPU-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>GPU</a></span></li><li><span><a href="#Multiple-Grover-Iterations" data-toc-modified-id="Multiple-Grover-Iterations-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Multiple Grover Iterations</a></span><ul class="toc-item"><li><span><a href="#Automated-plot-printing" data-toc-modified-id="Automated-plot-printing-4.3.1"><span class="toc-item-num">4.3.1&nbsp;&nbsp;</span>Automated plot printing</a></span></li></ul></li><li><span><a href="#Quantum-counting" data-toc-modified-id="Quantum-counting-4.4"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>Quantum counting</a></span></li></ul></li><li><span><a href="#Application-to-real-SNR's" data-toc-modified-id="Application-to-real-SNR's-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Application to real SNR's</a></span><ul class="toc-item"><li><span><a href="#Running-over-full-SNR-array-(broken-into-pieces)" data-toc-modified-id="Running-over-full-SNR-array-(broken-into-pieces)-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Running over full SNR array (broken into pieces)</a></span></li></ul></li></ul></div>

In [1]:
from qiskit import QuantumCircuit, assemble, Aer, QuantumRegister, ClassicalRegister, AncillaRegister, transpile
from qiskit.visualization import plot_bloch_multivector, plot_histogram, array_to_latex
import math
from math import log, ceil, floor
import plotly.express as px
import numpy as np
import timeit
import time
import os
from random import randrange

# Database functions

In [2]:
def create_oracle(v_reg, nb_check):
    qc = QuantumCircuit(v_reg + 1)
    
    a = len(bin(nb_check)[2:]) #This gives the length of binary value to encode 
    pos = 0
    for j in list(bin(nb_check)[2:])[::-1]+(v_reg-a)*['0']: #loops through the reverse ordered list composed of the binary encoding of the value
        if not int(j):
            qc.x(pos)
            
        pos += 1
        
    qc.mct(list(range(v_reg)),v_reg)
    
    pos = 0
    for j in list(bin(nb_check)[2:])[::-1]+(v_reg-a)*['0']: #loops through the reverse ordered list composed of the binary encoding of the value
        if not int(j):
            qc.x(pos)
            
        pos += 1
        
    return qc

create_oracle(4, 2).draw()

┌───┐     ┌───┐
q_0: ┤ X ├──■──┤ X ├
     └───┘  │  └───┘
q_1: ───────■───────
     ┌───┐  │  ┌───┐
q_2: ┤ X ├──■──┤ X ├
     ├───┤  │  ├───┤
q_3: ┤ X ├──■──┤ X ├
     └───┘┌─┴─┐└───┘
q_4: ─────┤ X ├─────
          └───┘

In [3]:
#This function takes in the number of qbits inside the index register and value register, as well as the array
# that works as database and ecodes those values automatically as gates.

#There are some extra x gates that are useless, and those will be removed in a next code update. They don't affect results.
def encode_qc(i_reg,v_reg,val_array):
    
    qc = QuantumCircuit(i_reg + v_reg)
    
    for step in range(len(val_array)):
        
        x_place = 0 #This variable is used to find where to put the NOT in the CNOT encoding
        a = len(bin(val_array[step])[2:]) #This gives the length of binary value to encode 
        
        for j in list(bin(val_array[step])[2:])[::-1]+(v_reg-a)*['0']: #loops through the reverse ordered list composed of the binary encoding of the value

            ### CONTROL X encoding ###
            if int(j):
                
                ### X encoding ###
                pos = 0
                b = len(bin(step)[2:])
                for i in list(bin(step)[2:])[::-1]+(i_reg-b)*['0']:

                    if not int(i) and int(j):
                        qc.x(pos)
                    pos += 1
                
                qc.mcx(list(range(i_reg)), i_reg + x_place)


                ### X encoding ###
                pos = 0
                b = len(bin(step)[2:])
                for i in list(bin(step)[2:])[::-1]+(i_reg-b)*['0']:

                    if not int(i):
                        qc.x(pos)
                    pos += 1
                    
#                 qc.barrier() #uncomment this is you want a more readable circuit, but then cannot be used as gate


            x_place += 1
    
    return qc

In [ ]:
encode_qc(3,4,[3,4,9]).draw(fold=-1)

In [ ]:
encode_qc(3,4,[3,4,9,13]).reverse_ops().draw(fold=-1)

# Comparator functions

In [4]:
def U_c():
    qc = QuantumCircuit(4)
    
    qc.x(1)
    qc.mcx([0,1],2, mode='noancilla')
    qc.x([0,1])
    qc.mcx([0,1],3, mode='noancilla')
    qc.x(0)
    
    return qc

In [ ]:
U_c().draw()

In [5]:
def n_comp_p1(n):
    qc = QuantumCircuit(5*n)
    
    u_c = U_c().to_gate()
    u_c.label = "U_c"
    
    for i in range(n):
        qc.append(u_c, range(i*5,i*5 + 4))
        
    for i in range(n-1):
        qc.x(i*5 + 2)
        qc.x(i*5 + 3)
        qc.mcx([i*5 + 2, i*5 + 3], i*5 + 4, mode='noancilla')
        qc.x(i*5 + 2)
        qc.x(i*5 + 3)
        
    for i in reversed(range(n-1)):
        qc.mcx([i*5 + 4,i*5 + 7], i*5 + 2, mode='noancilla')
        qc.mcx([i*5 + 4,i*5 + 8], i*5 + 3, mode='noancilla')
        
    return qc

In [ ]:
n_comp_p1(4).draw(fold=-1)

In [6]:
def n_comparator(n):
    qc = QuantumCircuit(5*n)
    
    c1 = n_comp_p1(n).to_gate()
    c1.label = "Comp_p1"
    
    qc.append(c1, range(5*n))
    
    qc.cx(3,5*n-1)
    
    qc.append(c1.reverse_ops(), range(5*n))
    
    return qc

n_comparator(3).draw()

┌───────────┐     ┌───────────┐
 q_0: ┤0          ├─────┤0          ├
      │           │     │           │
 q_1: ┤1          ├─────┤1          ├
      │           │     │           │
 q_2: ┤2          ├─────┤2          ├
      │           │     │           │
 q_3: ┤3          ├──■──┤3          ├
      │           │  │  │           │
 q_4: ┤4          ├──┼──┤4          ├
      │           │  │  │           │
 q_5: ┤5          ├──┼──┤5          ├
      │           │  │  │           │
 q_6: ┤6          ├──┼──┤6          ├
      │           │  │  │           │
 q_7: ┤7  Comp_p1 ├──┼──┤7  Comp_p1 ├
      │           │  │  │           │
 q_8: ┤8          ├──┼──┤8          ├
      │           │  │  │           │
 q_9: ┤9          ├──┼──┤9          ├
      │           │  │  │           │
q_10: ┤10         ├──┼──┤10         ├
      │           │  │  │           │
q_11: ┤11         ├──┼──┤11         ├
      │           │  │  │           │
q_12: ┤12         ├──┼──┤12         ├
      │           │  │  │           │
q_13: ┤13         ├──┼──┤13         ├
      │           │┌─┴─┐│           │
q_14: ┤14         ├┤ X ├┤14         ├
      └───────────┘└───┘└───────────┘

# Functions for final code

In [7]:
#This function takes in the quantum circuit to add the threshold, the size 
def set_thresh(qc, nb_i_bits, nb_v_bits, val):
    pos = 0
    a = len(bin(val)[2:])
    # This time no [::-1] (no reverse order), as the order of encoding is opposite to the system; instead we first add the 0s and then the binary nb
    for j in (nb_v_bits-a)*['0']+list(bin(val)[2:]): #loops through the reverse ordered list composed of the binary encoding of the value
        if int(j):
            qc.x(nb_i_bits+ pos*5)
            
        pos += 1

#This function creates an array of random values that are bellow the threshold indicated, and places nb_sol
#values of (thresh+1) randomely inside the array to act as solutions. This blinds the user to the database
#and they can use it afterwards as a double check on the correct functionong of the circuit. This also has
#the added benefit of making as big an array as the circuit can take. You can add additional solutions if 
#you wish, in the form of an array. those will also be placed randomly.
def array_creation(nb_qb_arr, thresh, nb_sol, extra_sol = []):
    if nb_sol + len(extra_sol) >= 2**(nb_qb_arr-1):
        raise NameError('Number of solution larger than N/2')
        
    data_arr = [randrange(thresh + 1) for x in range(2**nb_qb_arr)]

    i = 0
    while True:
        ind = randrange(len(data_arr))
        if data_arr[ind] < thresh and i < nb_sol:
            data_arr[ind] = thresh+1
            i+=1
        elif data_arr[ind] < thresh and len(extra_sol) != 0:
            data_arr[ind] = extra_sol.pop()
            
        if i >= nb_sol and len(extra_sol) == 0:
            break
            
    return data_arr

# Final Code

In [23]:
###################################### SETTING UP THE SEARCH ################################################
# In here you initialize the database that you will make the search on. This database has the structure of  #
# an array (data_arr). The size of this array and the maximum value that it contains will constrain the nb  #
# of qbits of the system to the closest highest power of two minus one(if array has length 9, the number of #
# qbits for the indices will be 4 -> 2^4 = 16; if max value is 16, we need 5 qbits to be able to hold the   #
# value).                                                                                                   #
# The threshold value dictates what you are looking for. By the nature of Grover's Algorithm, you should not#
# set a threshold values that marks more than 50% of the solutions, as then you will get inversed solutions #

nb_sol =3 #I want nb_sol solutions in my array
nb_qb_arr = 6 #I want 2^nb_qb_arr values in my array

thresh = 5
data_arr = array_creation(nb_qb_arr, thresh, nb_sol)
#############################################################################################################

#These numbers are calculated directly from the data array
n = ceil(log(len(data_arr), 2)) #number of index qbits
len_b =  ceil(log(max(data_arr)+1, 2))#length of the bitstrings to compare
m = 5*len_b #number of qbits needed for the comparision

i_reg = QuantumRegister(n, 'd') #Creates the data registry
val_reg = QuantumRegister(m-1, 't') #Creates the template registry
anc = QuantumRegister(1, 'ancilla') #Creates the ancillary registry for the oracle
cr = ClassicalRegister(n, 'c') #Creates the classical bit measurment output

qc = QuantumCircuit(i_reg, val_reg, anc, cr) #Makes the circuit with these qbits as input

encode = encode_qc(n,len_b,data_arr).to_gate()
encode.label = "Encoding database"

####### State initialization #########
qc.h(range(n))

#initiation of ancillary to |->
qc.x(anc)
qc.h(anc)

##### SETTING THE THRESHOLD ######

set_thresh(qc, n, len_b, thresh)

qc.barrier()

#ENCODING
#The arrays are made with list comprehentions to make something that looks like [0,1,2,n+16,n+11,n+6,n+1], 
#which connects the database (the first values) to the comparators (the values with n+...)
qc.append(encode, [x for x in range(n)]+[n+x*5+1 for x in reversed(range(len_b))])#had to reverse the order how the qbits were entered, as the two parts (database and conparator) work in reverse

comp_qc = n_comparator(len_b).to_gate()
comp_qc.label = "Comparator"

qc.append(comp_qc, range(n,n+m))

qc.append(encode.reverse_ops(), [x for x in range(n)]+[n+x*5+1 for x in reversed(range(len_b))])

qc.barrier()

#DIFFUSION OP
qc.h(range(n))
qc.x(range(n-1))

qc.x(n-1)
qc.h(n-1)
qc.mct(list(range(n-1)), n-1) 
qc.h(n-1)
qc.x(n-1)

qc.x(range(n-1))
qc.h(range(n))
# qc.barrier()

qc.measure([x for x in range(n)],cr)

# print(data_arr)
# c.draw(fold=-1)#output='latex_source',

## CPU

In [24]:
############# CIRCUIT SIMULATION #################
aer_sim = Aer.get_backend('aer_simulator')
aer_sim.set_options(device='CPU')
transpiled_qc = transpile(qc, aer_sim)

shots = 2**(n+7) #we repeat the simulation 2048 times
job = aer_sim.run(transpiled_qc, shots=shots)

############# CODE FOR PLOTTING ####################
hist = job.result().get_counts() #simulation output
sort_hist = sorted(hist.items()) #So that the plotting puts everything in the same increasing order of basis
n_hist = {k:v for k,v in sort_hist}

results = {'val':n_hist.keys(),'count':n_hist.values()} #change the formatting of the data to match plotly

fig = px.bar(results, x="val", y="count", text="count")
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(uniformtext_minsize=6, uniformtext_mode='show')
fig.show()

############## CODE FOR LISTING THE RESULTS #####################
trig = []
max_c = max(results['count'])
for i,j in zip(results['val'],results['count']):
    if int(j) > max_c-0.33*max_c: #only consideres values that are higher than one third less
        trig.append({'index_dec':int(i,2),'index_bin':i,'counts':j})

for t in trig:
    print(f"Index {t['index_dec']} holds the value {data_arr[t['index_dec']]}, which is above the chosen threshold.")
    
if not os.path.exists("plots/circuit_func"):
    os.makedirs("plots/circuit_func")
        
fig.write_image(f"plots/circuit_func/arr_len_{len(data_arr)}_hist.pdf",width=1000, height=600,)
fig.write_html(f"plots/circuit_func/arr_len_{len(data_arr)}_hist.html")

Index 12 holds the value 6, which is above the chosen threshold.
Index 32 holds the value 6, which is above the chosen threshold.
Index 42 holds the value 6, which is above the chosen threshold.


## GPU

This is only if you have a Nvidia graphics card installed. If so, change the cell back into a "Code" cell.

## Multiple Grover Iterations

In [8]:
#Function that combines the whole oracle + diffusion operator into one and returns it as a circuit
def GrovIt(data_arr, thresh):
    
    n = ceil(log(len(data_arr), 2)) #number of index qbits
    len_b =  ceil(log(max(data_arr)+1, 2))#length of the bitstrings to compare
    m = 5*len_b #number of qbits needed for the comparision

    i_reg = QuantumRegister(n, 'd') #Creates the data registry
    val_reg = QuantumRegister(m-1, 't') #Creates the template registry
    anc = QuantumRegister(1, 'ancilla') #Creates the ancillary registry for the oracle

    qc = QuantumCircuit(i_reg, val_reg, anc) #Makes the circuit with these qbits as input

    ##### SETTING THE THRESHOLD ######
    #part of the oracle, so left into the function
    set_thresh(qc, n, len_b, thresh)

    #ENCODING
    #The arrays are made with list comprehentions to make something that looks like [0,1,2,n+16,n+11,n+6,n+1], 
    #which connects the database (the first values) to the comparators (the values with n+...)
    
    encode = encode_qc(n,len_b,data_arr).to_gate()
    encode.label = "Encoding database"
    
    qc.append(encode, [x for x in range(n)]+[n+x*5+1 for x in reversed(range(len_b))])#had to reverse the order how the qbits were entered, as the two parts (database and conparator) work in reverse

    comp_qc = n_comparator(len_b).to_gate()
    comp_qc.label = "Comparator"

    qc.append(comp_qc, range(n,n+m))

    qc.append(encode.reverse_ops(), [x for x in range(n)]+[n+x*5+1 for x in reversed(range(len_b))])

    #DIFFUSION OP
    qc.h(range(n))
    qc.x(range(n-1))

    qc.x(n-1)
    qc.h(n-1)
    qc.mct(list(range(n-1)), n-1) 
    qc.h(n-1)
    qc.x(n-1)

    qc.x(range(n-1))
    qc.h(range(n))
    
    return qc

In [ ]:
###################################### SETTING UP THE SEARCH ################################################
nb_sol = 3 #I want nb_sol solutions in my array
nb_qb_arr = 3 #I want 2^nb_qb_arr values in my array

thresh = 5
data_arr = array_creation(nb_qb_arr, thresh, nb_sol)
#############################################################################################################

#These numbers are calculated directly from the data array
n = ceil(log(len(data_arr), 2)) #number of index qbits
len_b =  ceil(log(max(data_arr)+1, 2))#length of the bitstrings to compare
m = 5*len_b #number of qbits needed for the comparision

i_reg = QuantumRegister(n, 'd') #Creates the data registry
val_reg = QuantumRegister(m-1, 't') #Creates the template registry
anc = QuantumRegister(1, 'ancilla') #Creates the ancillary registry for the oracle
cr = ClassicalRegister(n, 'c') #Creates the classical bit measurment output

qc = QuantumCircuit(i_reg, val_reg, anc, cr) #Makes the circuit with these qbits as input

####### State initialization #########
qc.h(range(n))

#initiation of ancillary to |->
qc.x(anc)
qc.h(anc)

##### GROVER ITERATION ######
#THE THRESHOLD IS SET IN GROVER METHOD

grov_it = GrovIt(data_arr, thresh).to_gate()
grov_it.label = "Grover"

M = 1 #nb of Grov. it.
for i in range(M):
    qc.append(grov_it, list(range(n+m)))

qc.measure([x for x in range(n)],cr)
# qc.draw(fold=-1)

In [ ]:
############# CIRCUIT SIMULATION #################
aer_sim = Aer.get_backend('aer_simulator')
aer_sim.set_options(device='CPU')
transpiled_qc = transpile(qc, aer_sim)

shots = 2**(n+7) #we repeat the simulation 2048 times
job = aer_sim.run(transpiled_qc, shots=shots)

############# CODE FOR PLOTTING ####################
hist = job.result().get_counts() #simulation output
sort_hist = sorted(hist.items()) #So that the plotting puts everything in the same increasing order of basis
n_hist = {k:v for k,v in sort_hist}

results = {'val':n_hist.keys(),'count':n_hist.values()} #change the formatting of the data to match plotly

fig = px.bar(results, x="val", y="count", text="count")
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(uniformtext_minsize=6, uniformtext_mode='show')
fig.show()

############## CODE FOR LISTING THE RESULTS #####################
trig = []
max_c = max(results['count'])
for i,j in zip(results['val'],results['count']):
    if int(j) > max_c-0.33*max_c: #only consideres values that are higher than one third less
        trig.append({'index_dec':int(i,2),'index_bin':i,'counts':j})

for t in trig:
    print(f"Index {t['index_dec']} holds the value {data_arr[t['index_dec']]}, which is above the chosen threshold.")

### Automated plot printing

In the cell bellow, you can choose how many Grover iterations you want and it will automatically do from 0 to the number you want, saving the plots in two different formats, as well as saving the array, the indices and the threshold chosen into a file called array_thresh.py, in the folder plots/grov_its/ .

In [12]:
###################################### SETTING UP THE SEARCH ################################################
nb_times = 10 #up to how many grover it. you want to test

nb_sol = 7 #I want nb_sol solutions in my array
nb_qb_arr = 6 #I want 2^nb_qb_arr values in my array

thresh = 5
data_arr = array_creation(nb_qb_arr, thresh, nb_sol)
#############################################################################################################

ind = []
i = 0
for j in data_arr:
    if j > thresh:
        ind.append(i)
    i += 1
    
if not os.path.exists("plots/grov_its"):
        os.makedirs("plots/grov_its")
        
with open("plots/grov_its/array_thresh.py", 'w') as f:
    f.write(f'#Indices of values above {thresh}: {ind}. The values are {[data_arr[x] for x in ind]}\n')
    f.write(f'#The full array is {data_arr}\n\n')
    f.write(f'thresh = {thresh}\n')
    f.write(f'data_arr = {data_arr}\n')
    f.write(f'ind = {ind}\n')
    
M = 1
for i in range(nb_times):

    #These numbers are calculated directly from the data array
    n = ceil(log(len(data_arr), 2)) #number of index qbits
    len_b =  ceil(log(max(data_arr)+1, 2))#length of the bitstrings to compare
    m = 5*len_b #number of qbits needed for the comparision

    i_reg = QuantumRegister(n, 'd') #Creates the data registry
    val_reg = QuantumRegister(m-1, 't') #Creates the template registry
    anc = QuantumRegister(1, 'ancilla') #Creates the ancillary registry for the oracle
    cr = ClassicalRegister(n, 'c') #Creates the classical bit measurment output

    qc = QuantumCircuit(i_reg, val_reg, anc, cr) #Makes the circuit with these qbits as input

    ####### State initialization #########
    qc.h(range(n))

    #initiation of ancillary to |->
    qc.x(anc)
    qc.h(anc)

    ##### GROVER ITERATION ######
    #THE THRESHOLD IS SET IN GROVER METHOD

    grov_it = GrovIt(data_arr, thresh).to_gate()
    grov_it.label = "Grover"

    for i in range(M):
        qc.append(grov_it, list(range(n+m)))

    qc.measure([x for x in range(n)],cr)

    ############# CIRCUIT SIMULATION #################
    aer_sim = Aer.get_backend('aer_simulator')
    aer_sim.set_options(device='CPU')

    transpiled_qc = transpile(qc, aer_sim)
    shots = 2048 #we repeat the simulation 2048 times
    job = aer_sim.run(transpiled_qc)

    ############# CODE FOR PLOTTING ####################
    hist = job.result().get_counts() #simulation output
    sort_hist = sorted(hist.items()) #So that the plotting puts everything in the same increasing order of basis
    n_hist = {k:v for k,v in sort_hist}

    results = {'val':n_hist.keys(),'count':n_hist.values()} #change the formatting of the data to match plotly

    fig = px.bar(results, x="val", y="count", text="count")
    fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
    fig.update_layout(uniformtext_minsize=6, uniformtext_mode='show')
    fig.show()

    fig.write_image(f"plots/grov_its/r_arr_{M}_GrovIts.pdf",width=1600, height=800,)
    fig.write_html(f"plots/grov_its/r_arr_{M}_GrovIts.html")
    
    M+=1
    

## Quantum counting

We can make the code above into a function that encodes a single iteration of the Grover algorithm (oracle + diffusion):

And now create the quantum counting section:

In [9]:
def qft(n):
    """Creates an n-qubit QFT circuit"""
    circuit = QuantumCircuit(n)
    def swap_registers(circuit, n):
        for qubit in range(n//2):
            circuit.swap(qubit, n-qubit-1)
        return circuit
    def qft_rotations(circuit, n):
        """Performs qft on the first n qubits in circuit (without swaps)"""
        if n == 0:
            return circuit
        n -= 1
        circuit.h(n)
        for qubit in range(n):
            circuit.cp(np.pi/2**(n-qubit), qubit, n)
        qft_rotations(circuit, n)
    
    qft_rotations(circuit, n)
    swap_registers(circuit, n)
    return circuit

In [33]:
data_arr = [3,3,1,3,0,0,3,3,1,1,2,3,2,3]
thresh = 2
t = 8   # no. of counting qubits

grov_it = GrovIt(data_arr, thresh).to_gate()
grov_it.label = "Grover"
cgrov_it = grov_it.control()

qft_dagger = qft(t).to_gate().inverse()
qft_dagger.label = "QFT†"

n = ceil(log(len(data_arr), 2)) #number of index qbits
len_b =  ceil(log(max(data_arr)+1, 2))#length of the bitstrings to compare
m = 5*len_b #number of qbits needed for the comparision

c_reg = QuantumRegister(t, 'count')
i_reg = QuantumRegister(n, 'd') #Creates the data registry
val_reg = QuantumRegister(m-1, 't') #Creates the template registry
anc = QuantumRegister(1, 'ancilla') #Creates the ancillary registry for the oracle
cr = ClassicalRegister(t, 'c') #Creates the classical bit measurment output

qc = QuantumCircuit(c_reg, i_reg, val_reg, anc, cr) #Makes the circuit with these qbits as input

# Initialize all qubits to |+>
for qubit in range(t+n+m-1):
    qc.h(qubit)
    
# #initiation of ancillary to |->
qc.x(anc)
qc.h(anc)


# Begin controlled Grover iterations
iterations = 1
for qubit in range(t):
    for i in range(iterations):
        qc.append(cgrov_it, [qubit] + [*range(t, t+n+m)])
    iterations *= 2
    
# Do inverse QFT on counting qubits
qc.append(qft_dagger, range(t))

# Measure counting qubits
qc.measure(range(t), range(t))

# Display the circuit
# qc.draw(fold=-1)

In [34]:
# Execute and see results
aer_sim = Aer.get_backend('aer_simulator')
transpiled_qc = transpile(qc, aer_sim)
qobj = assemble(transpiled_qc)
job = aer_sim.run(qobj)
hist = job.result().get_counts()

results = {'val':hist.keys(),'count':hist.values()} #change the formatting of the data to match plotly

fig = px.bar(results, x="val", y="count", text="count")
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(uniformtext_minsize=6, uniformtext_mode='show')
fig.show()

if not os.path.exists("plots/circuit_func"):
    os.makedirs("plots/circuit_func")
        
fig.write_image(f"plots/circuit_func/quantum_counting_hist.pdf",width=1000, height=600,)
fig.write_html(f"plots/circuit_func/quantum_counting_hist.html")

/home/jonas/anaconda3/envs/QC_ENV/lib/python3.9/site-packages/qiskit/utils/deprecation.py:56: DeprecationWarning:

Using a qobj for run() is deprecated as of qiskit-aer 0.9.0 and will be removed no sooner than 3 months from that release date. Transpiled circuits should now be passed directly using `backend.run(circuits, **run_options).



In [36]:
measured_str = max(hist, key=hist.get)
measured_int = int(measured_str,2)
print("Register Output = %i" % measured_int)

theta = (measured_int/(2**t))*math.pi*2
print("Theta = %.5f" % theta)

N = 2**n
M = N * (math.sin(theta/2)**2)
print("No. of Solutions = %.1f" % (N-M))

k = math.pi/4*math.sqrt(N/(N-M))-0.5
print(f"The optimal number of iterations is: {k}")

Register Output = 0
Theta = 0.00000
No. of Solutions = 16.0
The optimal number of iterations is: 0.2853981633974483


In [37]:
max(hist, key=hist.get)

'00000000'

# Application to real SNR's

So let's try to apply this to the SNR array from the paper. For this, we will simply take the SNR file from that the github repo gives us an initialize the SNR array the same way:

In [ ]:
#Taken from the code of the paper
M = 2**17

snrs_ = np.load('snr_data/SNRs_signal_spins.npy')
if len(snrs_)>M:
    snrs = snrs_[::len(snrs_)//M]
    if len(snrs)>M:
        snrs = snrs[:M]
    elif len(snrs)<M:
        snrs__ = snrs_[1:][::len(snrs)//M][:M-len(snrs)]
        snrs = np.concatenate((snrs,snrs__))
else:
    snrs = snrs_

Because we are already cutting the array thanks to the code of the paper, we will have an exact power of two as array length. But let's check nontheless:

In [ ]:
print(f"SNR array is of length: {len(snrs)}")
pow2 = ceil(log(len(snrs), 2))
print(f"Closest upper power of two: {pow2} -> 2^{pow2} = {2**pow2}")

This is too big to run on my computer as is, as it requires too much computational power to create and simulate the circuit. So instead we will choose to take only the $2^9$ first values and try our algorithm on those. We also choose an SNR threshold of 6. We round the values in the array to the closest integer (in practice we should floor the values, but this is only a proof of concept).
Let's see what we should expect as an outcome:

In [ ]:
p = []
for i in np.rint(snrs.tolist()).astype(np.int64)[:2**11]:
    if i > 6:
        p.append(i)
        
print(p)

So these are the values that we will hopefully get as an output of the computation.

In [ ]:
###################################### SETTING UP THE SEARCH ################################################

data_arr = np.rint(snrs.tolist()).astype(np.int64)[:2**8]
thresh = 6

#############################################################################################################

#These numbers are calculated directly from the data array
n = ceil(log(len(data_arr), 2)) #number of index qbits
len_b =  ceil(log(max(data_arr)+1, 2))#length of the bitstrings to compare
m = 5*len_b #number of qbits needed for the comparision

i_reg = QuantumRegister(n, 'd') #Creates the data registry
val_reg = QuantumRegister(m-1, 't') #Creates the template registry
anc = QuantumRegister(1, 'ancilla') #Creates the ancillary registry for the oracle
cr = ClassicalRegister(n, 'c') #Creates the classical bit measurment output

qc = QuantumCircuit(i_reg, val_reg, anc, cr) #Makes the circuit with these qbits as input

encode = encode_qc(n,len_b,data_arr).to_gate()
encode.label = "Encoding database"

####### State initialization #########
qc.h(range(n))

#initiation of ancillary to |->
qc.x(anc)
qc.h(anc)

##### SETTING THE THRESHOLD ######

set_thresh(qc, n, len_b, thresh)

qc.barrier()

#ENCODING
#The arrays are made with list comprehentions to make something that looks like [0,1,2,n+16,n+11,n+6,n+1], 
#which connects the database (the first values) to the comparators (the values with n+...)
qc.append(encode, [x for x in range(n)]+[n+x*5+1 for x in reversed(range(len_b))])#had to reverse the order how the qbits were entered, as the two parts (database and conparator) work in reverse

comp_qc = n_comparator(len_b).to_gate()
comp_qc.label = "Comparator"

qc.append(comp_qc, range(n,n+m))

qc.append(encode.reverse_ops(), [x for x in range(n)]+[n+x*5+1 for x in reversed(range(len_b))])

qc.barrier()

#DIFFUSION OP
qc.h(range(n))
qc.x(range(n-1))

qc.x(n-1)
qc.h(n-1)
qc.mct(list(range(n-1)), n-1) 
qc.h(n-1)
qc.x(n-1)

qc.x(range(n-1))
qc.h(range(n))
qc.barrier()

qc.measure([x for x in range(n)],cr)

qc.draw(fold=-1)

In [ ]:
############# CIRCUIT SIMULATION #################
aer_sim = Aer.get_backend('aer_simulator')
aer_sim.set_options(device='CPU')
transpiled_qc = transpile(qc, aer_sim)

shots = 2**(n+7) #we repeat the simulation 2048 times
job = aer_sim.run(transpiled_qc, shots=shots)

############# CODE FOR PLOTTING ####################
hist = job.result().get_counts() #simulation output
sort_hist = sorted(hist.items()) #So that the plotting puts everything in the same increasing order of basis
n_hist = {k:v for k,v in sort_hist}

results = {'val':n_hist.keys(),'count':n_hist.values()} #change the formatting of the data to match plotly

fig = px.bar(results, x="val", y="count", text="count")
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(uniformtext_minsize=6, uniformtext_mode='show')
fig.show()

In [ ]:
############## CODE FOR LISTING THE RESULTS #####################
trig = []
max_c = max(results['count'])
for i,j in zip(results['val'],results['count']):
    if int(j) > max_c-0.33*max_c: #only consideres values that are higher than one third less
        trig.append({'index_dec':int(i,2),'index_bin':i,'counts':j})

for t in trig:
    print(f"Index {t['index_dec']} holds the value {data_arr[t['index_dec']]}, which is above the chosen threshold.")

## Running over full SNR array (broken into pieces)

In [ ]:
#BENCHMARKING VALUES#
val_qbit_nb = 4
block_pow = 4 #size of spliced block
max_loop = 32 #nb of loops to do
print_lim = 90 #print probability limit (this is the threshold at which the prob is high enough)
#####################

M = 2**17

snrs_ = np.load('snr_data/SNRs_signal_spins.npy')
if len(snrs_)>M:
    snrs = snrs_[::len(snrs_)//M]
    if len(snrs)>M:
        snrs = snrs[:M]
    elif len(snrs)<M:
        snrs__ = snrs_[1:][::len(snrs)//M][:M-len(snrs)]
        snrs = np.concatenate((snrs,snrs__))
else:
    snrs = snrs_

###################################### SETTING UP THE SEARCH ################################################
nb_break = 2**block_pow #size of blocks
broken_snrs = [snrs[i:i + nb_break] for i in range(0, len(snrs), nb_break)] #breaks the snr array into even blocks

data_arr_full = np.rint(broken_snrs).astype(np.int64) #rounds the values and transforms into integers

thresh = 6

with open('output_dicts.py', 'w') as f:
    f.write(f"# This file stores the values from the grover iteration")

with open('output_dicts.py', 'a') as f:
    f.write(
f'''
import numpy as np

M = 2**17

snrs_ = np.load('SNRs_signal_spins.npy')
if len(snrs_)>M:
    snrs = snrs_[::len(snrs_)//M]
    if len(snrs)>M:
        snrs = snrs[:M]
    elif len(snrs)<M:
        snrs__ = snrs_[1:][::len(snrs)//M][:M-len(snrs)]
        snrs = np.concatenate((snrs,snrs__))
else:
    snrs = snrs_

nb_break = 2**{block_pow} #size of blocks
broken_snrs = [snrs[i:i + nb_break] for i in range(0, len(snrs), nb_break)]
''')

#############################################################################################################
loop_nb = 0

with open('output_dicts.py', 'a') as f:
    f.write(f"dict_val = {{\n")

for data_arr in data_arr_full:
    
    print(f"\nLoop {loop_nb+1}; Indices {nb_break*loop_nb} to {nb_break*(loop_nb+1)}")

    #These numbers are calculated directly from the data array
    n = ceil(log(len(data_arr), 2)) #number of index qbits
    
    if ceil(log(max(data_arr)+1, 2)) > val_qbit_nb:
        len_b = ceil(log(max(data_arr)+1, 2))
    else:
        len_b = val_qbit_nb
    
#     len_b =  ceil(log(max(data_arr)+1, 2))#length of the bitstrings to compare
    m = 5*len_b #number of qbits needed for the comparision

    i_reg = QuantumRegister(n, 'd') #Creates the data registry
    val_reg = QuantumRegister(m-1, 't') #Creates the template registry
    anc = QuantumRegister(1, 'ancilla') #Creates the ancillary registry for the oracle
    cr = ClassicalRegister(n, 'c') #Creates the classical bit measurment output

    qc = QuantumCircuit(i_reg, val_reg, anc, cr) #Makes the circuit with these qbits as input

    encode = encode_qc(n,len_b,data_arr).to_gate()
    encode.label = "Encoding database"

    ####### State initialization #########
    qc.h(range(n))

    #initiation of ancillary to |->
    qc.x(anc)
    qc.h(anc)

    ##### SETTING THE THRESHOLD ######

    set_thresh(qc, n, len_b, thresh)

    qc.barrier()

    #ENCODING
    #The arrays are made with list comprehentions to make something that looks like [0,1,2,n+16,n+11,n+6,n+1],
    #which connects the database (the first values) to the comparators (the values with n+...)
    qc.append(encode, [x for x in range(n)]+[n+x*5+1 for x in reversed(range(len_b))])#had to reverse the order how the qbits were entered, as the two parts (database and conparator) work in reverse

    comp_qc = n_comparator(len_b).to_gate()
    comp_qc.label = "Comparator"

    qc.append(comp_qc, range(n,n+m))

    qc.append(encode.reverse_ops(), [x for x in range(n)]+[n+x*5+1 for x in reversed(range(len_b))])

    qc.barrier()

    #DIFFUSION OP
    qc.h(range(n))
    qc.x(range(n-1))

    qc.x(n-1)
    qc.h(n-1)
    qc.mct(list(range(n-1)), n-1)
    qc.h(n-1)
    qc.x(n-1)

    qc.x(range(n-1))
    qc.h(range(n))
    qc.barrier()

    qc.measure([x for x in range(n)],cr)

    
    ############# CIRCUIT SIMULATION #################
    aer_sim = Aer.get_backend('aer_simulator')
    aer_sim.set_options(device='CPU')
    transpiled_qc = transpile(qc, aer_sim)

    shots = 2**(n+7) #we repeat the simulation 2048 times
    job = aer_sim.run(transpiled_qc, shots=shots)


    hist = job.result().get_counts() #simulation output
    sort_hist = sorted(hist.items()) #So that the plotting puts everything in the same increasing order of basis
    n_hist = {k:v for k,v in sort_hist}

    results = {'val':n_hist.keys(),'count':n_hist.values()}

    trig = []
    max_c = max(results['count'])
    for i,j in zip(results['val'],results['count']):
        if int(j) > max_c-0.33*max_c: #only consideres values that are higher than one third less
            trig.append({'index_dec':int(i,2),'index_bin':i,'val':data_arr[int(i,2)],'counts':j})
            
    print(f'Values found: {len(trig)}')
    
#     for t in trig:
#         print(f"Index {t['index_dec']} holds the value {data_arr[t['index_dec']]}, which is above the chosen threshold.")

    f = open('output_dicts.py', 'a')
    f.write(f"{loop_nb}:{trig}, # Indices {nb_break*loop_nb} to {nb_break*(loop_nb+1)}\n")
    f.close()

    #To limit how many iterations. Comment if want to do the whole array.
#     if loop_nb == max_loop-1:
#         break

    loop_nb += 1

with open('output_dicts.py', 'a') as f:
    f.write("}\n\n")
    f.write(
'''
index_array = []
for i in dict_val:
    for sol in dict_val[int(i)]:
        index_array.append(i*nb_break + sol['index_dec'])
        
def print_snr_abv(value):
    for i in index_array:
        if snrs[i] > value:
            print(i,snrs[i])


for i in index_array:
    if snrs[i] > 12:
        print(f"SNR Index: {i};  \t\tSNR value: {round(snrs[i],3)}")
'''
    )


Recovering all the SNRs found with QC

In [ ]:
#had to hand modify some things in the file before this was possible
from output_dicts import dict_val

Finding classically all matching SNRs:

In [ ]:
p = []
for i in snrs[:2**4*1549]: #the size of the breaking times the number of loops made before crash
    if np.rint(i) > 6:
        p.append(round(i,3))
        print(round(i,3))

Testing that the QC found SNRs are an exact match to the ones found classically:

In [ ]:
j = 0
found_all = True
for i in index_array:
    if round(snrs[i],3) == p[j]:
        pass
    else:
        found_all = False
        print(i, round(snrs[i],3), p[j])
    j+=1
print(f"The statement that the snrs above threshold are all found is {found_all}.")